In [1]:
import os
from collections import defaultdict
import re

# ⚠️ 重要: 以下のファイルパスを、あなたの環境のWordNetデータファイル（dictフォルダ内など）の正しいパスにすべて置き換えてください。
WORDNET_FILE_PATHS = {
    'noun': 'wordnet/dict/index.noun', 
    'verb': 'wordnet/dict/index.verb',
    'adj': 'wordnet/dict/index.adj',
    'adv': 'wordnet/dict/index.adv'
}

# WordNetの生のindexファイルには、データ行の前に著作権情報などのヘッダー行があります。
# WordNet 3.0の一般的なヘッダー行数（スキップする行数）を設定します。
HEADER_LINES_TO_SKIP = 29 # WordNet 3.0の場合の目安

    
   

In [2]:

# --- フィルタリング関数 ---

def clean_and_filter_lemma(lemma, exclusion_counts, raw_lemmas_set):
    """
    単語をクリーニングし、主要な単語として適切かフィルタリングする。
    
    Args:
        lemma (str): WordNetから抽出された生の単語（レマ）。
        exclusion_counts (defaultdict): 除外理由を記録するための辞書。
        raw_lemmas_set (set): フィルタリング前の全てのユニークな単語を記録する集合。
        
    Returns:
        str or None: フィルタリングを通過した単語（小文字化）または None。
    """
    
    # 処理前の集合に生のレマ（WordNet形式）を追加
    # raw_lemmas_set.add(lemma) は、ファイル単位の重複を除外するため、
    # 呼び出し元（extract_lemmas_with_filtering）で行うのが適切です。
    
    # 1. 複合語（アンダースコア）の除外
    if '_' in lemma:
        exclusion_counts['Compound_Word'] += 1
        return None
        
    # 2. 固有名詞/大文字の除外 (全て小文字であること)
    if not lemma.islower():
        exclusion_counts['Proper_Noun_or_Capitalized'] += 1
        return None
        
    cleaned_lemma = lemma.lower()
    
    # 3. 数字・記号の除外 (アルファベットとハイフンのみを許可)
    if re.search(r'[^a-z-]', cleaned_lemma):
        exclusion_counts['Symbol_or_Numeric'] += 1
        return None
        
    # 4. 短すぎる単語の除外 (長さが3文字未満のものを除去)
    min_word_length = 3
    if len(cleaned_lemma) < min_word_length:
        exclusion_counts['Too_Short'] += 1
        return None
        
    return cleaned_lemma

# --- 抽出関数（フィルタリングと計上を適用） ---

def extract_lemmas_with_filtering(file_path, exclusion_counts, raw_lemmas_set):
    """
    indexファイルから単語を抽出し、フィルタリングを行う。
    生の単語を raw_lemmas_set に追加し、フィルタリング後の単語を返す。
    """
    if not os.path.exists(file_path):
        print(f"[警告] ファイルが見つかりません: {file_path}。スキップします。")
        return set()

    filtered_lemmas = set()
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for _ in range(HEADER_LINES_TO_SKIP):
                next(f, None)
                
            for line in f:
                line = line.strip()
                if not line or line.startswith(' '):
                    continue
                    
                parts = line.split()
                if parts:
                    raw_lemma = parts[0]
                    
                    # 💡 生の単語を即座に集合に追加（全ファイルの重複を避けるため）
                    raw_lemmas_set.add(raw_lemma)
                    
                    # フィルタリング関数を適用
                    cleaned_lemma = clean_and_filter_lemma(raw_lemma, exclusion_counts, raw_lemmas_set)
                    
                    if cleaned_lemma:
                        filtered_lemmas.add(cleaned_lemma)
                        
    except Exception as e:
        print(f"[エラー] ファイル {file_path} の処理中にエラーが発生しました: {e}")
        return set()

    return filtered_lemmas

# --- メイン処理 ---
raw_lemmas_set = set() # 👈 処理前のユニークな単語集合（WordNet形式）
all_unique_lemmas = set() # フィルタリング後のユニークな単語集合
exclusion_counts = defaultdict(int)
processed_files = 0

print("WordNet Indexファイルの処理を開始します...")

for pos, path in WORDNET_FILE_PATHS.items():
    if os.path.exists(path):
        print(f"\n--- {pos.upper()}ファイル ({os.path.basename(path)}) を処理中 ---")
        
        # raw_lemmas_set には生の単語が追加され、filtered_lemmas にはフィルタリング後の単語が追加される
        filtered_lemmas = extract_lemmas_with_filtering(path, exclusion_counts, raw_lemmas_set)
        
        all_unique_lemmas.update(filtered_lemmas)
        processed_files += 1
        print(f"-> フィルタリング後のユニークな単語数 (このファイル由来): {len(filtered_lemmas)}")

# --- 結果の表示 ---
print("\n" + "=" * 50)
print("📚 処理結果の概要")
print("=" * 50)

# 処理前の集合サイズ（全ファイルからのユニークな生単語数）
total_raw_unique_lemmas = len(raw_lemmas_set)
# フィルタリング後の集合サイズ
total_filtered_unique_lemmas = len(all_unique_lemmas)

# 除去された単語数は、生の集合からフィルタリング後の集合を引いて算出します。
# 除去された単語の総数 = 生のユニーク数 - フィルタリング後のユニーク数
total_removed = total_raw_unique_lemmas - total_filtered_unique_lemmas


print(f"| 処理前のユニークな生単語総数: {total_raw_unique_lemmas:,} 個")
print(f"| フィルタリング後のユニークな主要単語総数: {total_filtered_unique_lemmas:,} 個")
print("-" * 50)
print(f"| 最終的に除去されたユニーク単語総数: {total_removed:,} 個")
print("-" * 50)

print("\n--- 除去理由の内訳（重複カウントあり）---")
# 除去理由とその単語数のリスト
removal_reasons = {
    "Compound_Word": "複合語 (アンダースコア '_')",
    "Proper_Noun_or_Capitalized": "固有名詞/大文字を含む",
    "Symbol_or_Numeric": "記号や数字を含む",
    "Too_Short": "短すぎる単語 (3文字未満)"
}

# フィルタリング関数内でカウントされた数を表示
for key, description in removal_reasons.items():
    count = exclusion_counts[key]
    print(f"| {description:<25}: {count:,} 個")

print("=" * 50)

# フィルタリング後の単語の例を表示
unique_word_list = sorted(list(all_unique_lemmas))
print("\nフィルタリング後のユニークな単語の例 (最初の20個):")
for i, word in enumerate(unique_word_list[:20]):
    print(f"{i+1}. {word}")

WordNet Indexファイルの処理を開始します...

--- NOUNファイル (index.noun) を処理中 ---
-> フィルタリング後のユニークな単語数 (このファイル由来): 56955

--- VERBファイル (index.verb) を処理中 ---
-> フィルタリング後のユニークな単語数 (このファイル由来): 8691

--- ADJファイル (index.adj) を処理中 ---
-> フィルタリング後のユニークな単語数 (このファイル由来): 20718

--- ADVファイル (index.adv) を処理中 ---
-> フィルタリング後のユニークな単語数 (このファイル由来): 3736

📚 処理結果の概要
| 処理前のユニークな生単語総数: 147,478 個
| フィルタリング後のユニークな主要単語総数: 82,368 個
--------------------------------------------------
| 最終的に除去されたユニーク単語総数: 65,110 個
--------------------------------------------------

--- 除去理由の内訳（重複カウントあり）---
| 複合語 (アンダースコア '_')        : 64,375 個
| 固有名詞/大文字を含む              : 178 個
| 記号や数字を含む                 : 456 個
| 短すぎる単語 (3文字未満)           : 358 個

フィルタリング後のユニークな単語の例 (最初の20個):
1. a-bomb
2. a-horizon
3. a-line
4. a-list
5. a-ok
6. a-okay
7. a-one
8. a-team
9. aaa
10. aachen
11. aah
12. aalborg
13. aalii
14. aalst
15. aalto
16. aar
17. aardvark
18. aardwolf
19. aare
20. aarhus


In [3]:
# word2vec(googlenews-vectors-negative300.bin)の単語リストとの一致を確認する

file_path = "../22_word_label/processed_data/w2v_words_full.txt"
with open(file_path, 'r', encoding='utf-8') as f:
    w2v_words = set(line.strip() for line in f if line.strip())

print(f"length of w2v_words: {len(w2v_words)}") # 3000000

# common: 処理前のWordNet
common_words = raw_lemmas_set.intersection(w2v_words)
print(f"Number of common words between raw WordNet lemmas and Word2Vec vocabulary: {len(common_words)}")
# common: 処理後のWordNet
common_words = all_unique_lemmas.intersection(w2v_words)
print(f"Number of common words between WordNet filtered lemmas and Word2Vec vocabulary: {len(common_words)}")



length of w2v_words: 2999996
Number of common words between raw WordNet lemmas and Word2Vec vocabulary: 54623
Number of common words between WordNet filtered lemmas and Word2Vec vocabulary: 48272


In [ ]:
# 複合語の例(wordnet)を表示
print("\n複合語の例:")
compound_examples = [lemma for lemma in raw_lemmas_set if '_' in lemma]
for example in compound_examples[:20]:
    print(example)


複合語の例:
golden_state
soccer_field
eugene_ionesco
ischemic_stroke
bull_pine
finnan_haddock
scrutin_uninominal_voting_system
podocarpus_amara
grimm's_law
cichorium_endivia
pumping_station
genus_planera
news_leak
corn_spurrey
air_combat_command
ho_chi_minh_city
bok_choy
lacewing_fly
shiny_lyonia
survivorship_annuity


In [17]:
# 複合語の例(word2vec)を表示
import random


print("\nWord2Vecの複合語の例:")
compound_w2v_examples = [word for word in w2v_words if '_' in word]
# ランダムに取ってきたいくつかを表示
for i in random.sample(compound_w2v_examples, 100):
    print(i)


Word2Vecの複合語の例:
Dimetapp_Pediacare_Robitussin_Triaminic
Sounding_gong
Jon_Sioredas
By_Ruth_Mosalski
patented_windmill_windup
Philip_Quast
Understanding_Malaysia_Ikim
CPSC_Posts
Earnings_Preview_BJ_Wholesale
ankle_brachial
expedition_outfitter
Escorts_Balanced_Fund
Amy_Shailene_Woodley
Rosanna_Capolingua
fecal_occult
Chung_Shing
Once_Upon_Mattress
Miraculins_Announces
Aldon_Smith_DE
Burdale_Financial
Shelly_Gobin
SUTENT_sunitinib_malate
PureSpectrum_dimmable_CFL
Natural_Gas_CHNG
toronto_canada
David_Engelthaler_epidemiologist
AMEX_TIV
Mohammed_Yousri
By_Barney_Lerten
Inc._NASDAQ_CEBK
Ashoura_celebrations
HOME_COMFORT
NOAA_Geostationary_Operational
nick_Los_Cabos
Point_Defiance
Xun_Zhou
іn_аƖƖ
drought_tolerant_crops
Zahid_Gulfam
lateral_agility
CHARLOTTE_CHAMBERS
Tomi_Mine
wholly_owned_subsidiary_ZenZuu
Sheriff_Lonny_Pulkrabek
Ifakara_Health_Institute
Bermuda_shorts
Beaverton_Tigard_Tualatin
Attorney_Kurt_Alme
Mahindra_Rajapakse
WEAU_TV
Chunxiao_fields
lateral_ligaments
Cardiolite_®_Kit

In [21]:
# look_upを探してみる
# wordnet
print("\n'look_up' in WordNet lemmas:", 'look_up' in raw_lemmas_set)

# word2vec
print("'look_up' in Word2Vec vocabulary:", 'look_up' in w2v_words)

# 形式を色々変えてみて
print("'Look_up' in Word2Vec lemmas:", 'Look_up' in w2v_words)
print("'LOOK_UP' in Word2Vec lemmas:", 'LOOK_UP' in w2v_words)
print("'look-up' in Word2Vec lemmas:", 'look-up' in w2v_words)
print("'lookUp' in Word2Vec lemmas:", 'lookUp' in w2v_words)
print("'lookup' in Word2Vec lemmas:", 'lookup' in w2v_words) 


'look_up' in WordNet lemmas: True
'look_up' in Word2Vec vocabulary: False
'Look_up' in Word2Vec lemmas: False
'LOOK_UP' in Word2Vec lemmas: True
'look-up' in Word2Vec lemmas: False
'lookUp' in Word2Vec lemmas: False
'lookup' in Word2Vec lemmas: True


In [23]:
# 共通部分

# --- 1. Word2Vec語彙の読み込みと正規化 ---

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        # Word2Vecの語彙を全て小文字に正規化してセットに格納
        w2v_words_normalized = set(line.strip().lower() for line in f if line.strip())
    
    print(f"length of w2v_words_normalized: {len(w2v_words_normalized)}")

except FileNotFoundError:
    print(f"エラー: Word2Vecファイルが見つかりません: {file_path}")
    w2v_words_normalized = set()

# --- 2. WordNetレマの多角的候補生成 ---

wn_candidate_set = set()
total_candidates = 0

for raw_lemma in raw_lemmas_set:
    # 全て小文字に変換（Word2Vec語彙も小文字化されているため）
    lower_lemma = raw_lemma.lower()
    total_candidates += 1 # 生のレマ数をカウント
    
    # 候補リストを初期化
    candidates = set()
    
    # --- 候補生成 ---
    
    # 1. 連結小文字 (例: look_up -> lookup, run-in -> runin)
    #    アンダースコアとハイフンを除去
    candidates.add(lower_lemma.replace('_', '').replace('-', ''))
    
    # 2. スペース区切り小文字 (例: look_up -> look up, run-in -> run in)
    candidates.add(lower_lemma.replace('_', ' ').replace('-', ' '))
    
    # 3. 生形式小文字 (例: look_up -> look_up)
    #    Word2Vecがアンダースコアをトークンとして残している場合に対応
    candidates.add(lower_lemma)
    
    # 4. 構成単語の個別抽出
    #    '_', '-', ' ' のいずれかで分割される単語があれば、それらを個別に追加
    if '_' in lower_lemma or '-' in lower_lemma:
        parts = re.split(r'[_\-\s]+', lower_lemma)
        for part in parts:
            if part:
                candidates.add(part)
    
    # 候補を全体の集合に追加
    wn_candidate_set.update(candidates)
    
print(f"WordNetから生成されたユニークな候補総数: {len(wn_candidate_set)}")


# --- 3. 共通部分の計算 ---

# 共通集合 = (生成されたWordNet候補全て) ∩ (正規化されたWord2Vec語彙)
common_words = wn_candidate_set.intersection(w2v_words_normalized)

# --- 結果の表示 ---
print("\n" + "=" * 50)
print("📚 共通語彙の計算結果")
print("=" * 50)
print(f"WordNet 生レマ数 (入力): {len(raw_lemmas_set):,} 個")
print(f"WordNet 候補総数 (生成): {len(wn_candidate_set):,} 個")
print("-" * 50)
print(f"共通する単語の数 (最大限一致): {len(common_words):,} 個")
print("=" * 50)

# 例として、取得した共通語彙の一部を表示
if common_words:
    print("\n共通する単語の例 (最初の10個):")
    # 複合語だったものや、固有名詞だったものが小文字で一致している可能性がある
    for i, word in enumerate(sorted(list(common_words))[:10]):
        print(f"{i+1}. {word}")

length of w2v_words_normalized: 2702146
WordNetから生成されたユニークな候補総数: 294445

📚 共通語彙の計算結果
WordNet 生レマ数 (入力): 147,478 個
WordNet 候補総数 (生成): 294,445 個
--------------------------------------------------
共通する単語の数 (最大限一致): 78,600 個

共通する単語の例 (最初の10個):
1. 's
2. 0
3. 1
4. 1st
5. 1st_class
6. 2
7. 2d
8. 2nd
9. 3
10. 3d


# Word2Vec

In [6]:
# --- フィルタリング関数 ---
import csv

def clean_and_filter_w2v_word(word, exclusion_counts):
    """
    Word2Vecの単語をクリーニングし、主要な単語として適切かフィルタリングする。
    """
    
    # 1. 複合語（アンダースコア '_') の除外
    if '_' in word:
        exclusion_counts['Compound_Word'] += 1
        return None

    # 2. 固有名詞/大文字の除外 (完全に小文字でない単語)
    if not word.islower():
        exclusion_counts['Proper_Noun_or_Capitalized'] += 1
        return None

    # ここからは小文字に正規化された単語として扱う
    cleaned_word = word.lower() 
    
    # 3. 非常に短い単語の除去 (3文字未満)
    min_word_length = 3
    if len(cleaned_word) < min_word_length:
        exclusion_counts['Too_Short'] += 1
        return None
    
    # 4. アンダースコアのみの単語の除去 (アンダースコアは既に除外されているが、念のため)
    if all(c == '_' for c in cleaned_word):
        exclusion_counts['Underscore_Only'] += 1
        return None
        
    # 5. 記号・数字の除去 (アルファベットとハイフン以外)
    # 複合語は既に除外されたため、ここでは残ったハイフンやノイズをチェック
    if re.search(r'[^a-z\-]', cleaned_word):
        exclusion_counts['Symbol_or_Numeric'] += 1
        return None
        
    return cleaned_word

# --- メイン処理 ---
raw_w2v_words_set = set(w2v_words) 
filtered_w2v_words = set()
exclusion_counts = defaultdict(int)

print("Word2Vec語彙リストのクリーンアップを開始します...")

for word in raw_w2v_words_set:
    # フィルタリング適用
    cleaned_word = clean_and_filter_w2v_word(word, exclusion_counts)
    
    if cleaned_word:
        # フィルタリング後の単語は小文字化されている
        filtered_w2v_words.add(cleaned_word)

# --- 結果の表示 ---
print("\n" + "=" * 50)
print("📚 Word2Vec語彙クリーンアップ結果（複合語・大文字除去あり）")
print("=" * 50)

# 生のユニークな単語数を計算
total_raw_unique = len(raw_w2v_words_set)
# フィルタリング後のユニークな単語数
total_filtered_unique = len(filtered_w2v_words)
# 除去された単語の総数
total_removed = total_raw_unique - total_filtered_unique

print(f"| 処理前のユニークな生語彙総数: {total_raw_unique:,} 個")
print(f"| フィルタリング後のユニークな単語総数: {total_filtered_unique:,} 個")
print("-" * 50)
print(f"| 最終的に除去されたユニーク単語総数: {total_removed:,} 個")
print("-" * 50)

print("\n--- 除去理由の内訳（エントリベース）---")
# 除去理由とその単語数のリスト
removal_reasons = {
    "Compound_Word": "複合語 (アンダースコア '_')",
    "Proper_Noun_or_Capitalized": "固有名詞/大文字を含む",
    "Too_Short": "短すぎる単語 (3文字未満)",
    "Underscore_Only": "アンダースコアのみ",
    "Symbol_or_Numeric": "記号や数字を含む"
}

# フィルタリング関数内でカウントされた数を表示
for key, description in removal_reasons.items():
    count = exclusion_counts[key]
    print(f"| {description:<35}: {count:,} 個")

print("=" * 50)

# フィルタリング後の単語の例を表示
sample_list = random.sample(list(filtered_w2v_words), min(20, total_filtered_unique))
print("\nフィルタリング後のユニークな単語の例 (20個):")
for i, word in enumerate(sorted(sample_list)):
    print(f"{i+1}. {word}")

Word2Vec語彙リストのクリーンアップを開始します...

📚 Word2Vec語彙クリーンアップ結果（複合語・大文字除去あり）
| 処理前のユニークな生語彙総数: 2,999,996 個
| フィルタリング後のユニークな単語総数: 154,429 個
--------------------------------------------------
| 最終的に除去されたユニーク単語総数: 2,845,567 個
--------------------------------------------------

--- 除去理由の内訳（エントリベース）---
| 複合語 (アンダースコア '_')                  : 2,070,977 個
| 固有名詞/大文字を含む                        : 746,281 個
| 短すぎる単語 (3文字未満)                     : 1,201 個
| アンダースコアのみ                          : 0 個
| 記号や数字を含む                           : 27,108 個


NameError: name 'random' is not defined

# word2vecのフィルタと保存

In [7]:
# --- フィルタリング関数 (クリーンアップ用) ---
import csv
# 単なるクリーンアップでは複合語や大文字は除去せず、小文字化とノイズ除去のみを行います。
def simple_clean_w2v_word(word):
    """ノイズと短すぎる単語を除去し、小文字に正規化する。複合語は保持。"""
    
    # 1. 非常に短い単語の除去 (3文字未満)
    min_word_length = 3
    if len(word) < min_word_length:
        return None
    
    # 2. アンダースコアのみの単語の除去
    if all(c == '_' for c in word):
        return None
        
    # 3. 記号・数字の除去 (アルファベット、ハイフン、アンダースコア以外)
    # W2V語彙は複合語や略語としてこれらの記号を含むことが多いため、慎重にチェック
    if re.search(r'[^a-zA-Z0-9_\-]', word):
        return None
        
    # 4. 小文字への正規化
    cleaned_word = word.lower()
    
    return cleaned_word

# --- 共通部分計算関数 (WordNetとの比較用) ---

def get_common_and_cleaned_words(raw_w2v_list, raw_wn_set):
    """
    WordNetとWord2Vecの語彙の共通部分を計算する。
    WordNetのレマを複数の形式に変換し、Word2Vecのクリーンアップされた語彙と比較する。
    """
    
    # 1. Word2Vec語彙のクリーンアップと小文字化
    # W2V語彙は、単なるクリーンアップを適用して集合を作成します。
    w2v_words_cleaned = set(filter(None, (simple_clean_w2v_word(w) for w in raw_w2v_list)))

    # 2. WordNetレマの多角的候補生成 (小文字、連結、構成要素など)
    wn_candidate_set = set()
    for raw_lemma in raw_wn_set:
        lower_lemma = raw_lemma.lower()
        
        # 候補生成
        candidates = set()
        
        # a. 連結小文字 (例: look_up -> lookup, run-in -> runin)
        candidates.add(lower_lemma.replace('_', '').replace('-', ''))
        
        # b. スペース区切り小文字 (例: look_up -> look up, run-in -> run in)
        candidates.add(lower_lemma.replace('_', ' ').replace('-', ' '))
        
        # c. 生形式小文字 (例: look_up -> look_up)
        candidates.add(lower_lemma)
        
        # d. 構成単語の個別抽出 (例: big_deal -> big, deal)
        if '_' in lower_lemma or '-' in lower_lemma:
            parts = re.split(r'[_\-\s]+', lower_lemma)
            for part in parts:
                if part and simple_clean_w2v_word(part): # 単語がクリーンであるかチェック
                    candidates.add(part)
        
        wn_candidate_set.update(candidates)
        
    # 3. 共通部分の計算
    # 共通集合 = (生成されたWordNet候補全て) ∩ (クリーンアップされたWord2Vec語彙)
    common_words = wn_candidate_set.intersection(w2v_words_cleaned)
    
    # 共通語彙と、クリーンアップされたW2V語彙を両方返します
    return w2v_words_cleaned, common_words

# --- メイン処理と保存 ---

def save_indexed_word_list(word_set, filename_prefix):
    """
    単語集合をソートし、インデックス付きでCSVファイルに保存する。
    """
    output_dir = 'processed_data'
    os.makedirs(output_dir, exist_ok=True)
    
    word_list_to_save = sorted(list(word_set))
    output_path = os.path.join(output_dir, f'{filename_prefix}_indexed_words.csv')
    
    try:
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['index', 'word'])
            
            for index, word in enumerate(word_list_to_save):
                writer.writerow([index, word])
                
        print(f"✅ 保存成功 ({filename_prefix}): {output_path}")
        print(f"   保存された単語数: {len(word_list_to_save):,} 個")
        return True
    except Exception as e:
        print(f"❌ エラー: {filename_prefix} の保存中に問題が発生しました: {e}")
        return False

# --- 実行 ---
print("Word2Vec語彙とWordNetレマの処理を開始します...")

# 1. 共通語彙とクリーンアップされたW2V語彙を取得
w2v_cleaned_set, common_words_set = get_common_and_cleaned_words(w2v_words, raw_lemmas_set)

# 2. 単なるクリーンアップされたW2V語彙の保存
print("\n--- 1. W2Vクリーンアップリストの保存 ---")
save_indexed_word_list(w2v_cleaned_set, 'w2v_cleaned')

# 3. WordNetとの共通部分の保存
print("\n--- 2. WN/W2V共通語彙リストの保存 ---")
save_indexed_word_list(common_words_set, 'wn_w2v_common')

print("\n--- 完了 ---")

Word2Vec語彙とWordNetレマの処理を開始します...

--- 1. W2Vクリーンアップリストの保存 ---
✅ 保存成功 (w2v_cleaned): processed_data\w2v_cleaned_indexed_words.csv
   保存された単語数: 2,480,159 個

--- 2. WN/W2V共通語彙リストの保存 ---
✅ 保存成功 (wn_w2v_common): processed_data\wn_w2v_common_indexed_words.csv
   保存された単語数: 77,936 個

--- 完了 ---


# WordNet

In [27]:
# 再帰回数の設定
RECURSION_LIMIT = 1000000
import sys
sys.setrecursionlimit(RECURSION_LIMIT)

In [ ]:
import nltk
from nltk.corpus import wordnet as wn

# WordNetデータがダウンロードされていない場合は実行してください
# nltk.download('wordnet')

def extract_hypernym_tree(synset, depth=3, max_hyponyms=5, current_depth=0, node_list=None, edge_list=None):
    """
    特定のSynsetを起点に、ハイパーニミー（上位概念）とヒポニミー（下位概念）の
    階層構造を再帰的に抽出し、ノードとエッジのリストを生成する。
    
    Args:
        synset (Synset): 現在処理中のシノニムセット。
        depth (int): 階層をたどる最大深度。
        max_hyponyms (int): 各階層で抽出する下位概念の最大数（可視化の複雑さを軽減するため）。
    """
    if node_list is None:
        node_list = []
    if edge_list is None:
        edge_list = []
    
    if current_depth > depth:
        return node_list, edge_list

    # ノード名の生成: Synset名 (定義の最初のレマ)
    node_id = synset.name()
    node_label = synset.lemmas()[0].name().replace('_', ' ')
    
    if node_id not in [n['id'] for n in node_list]:
        node_list.append({
            'id': node_id,
            'label': node_label,
            'depth': current_depth
        })

    # --- 1. 上位概念（ハイパーニミー）を抽出 ---
    # 通常、単一パス（または少ないパス）の上位概念をたどる
    hypernyms = synset.hypernyms()
    for hypernym in hypernyms:
        hypernym_id = hypernym.name()
        # エッジを追加 (下位 -> 上位)
        if (node_id, hypernym_id) not in edge_list:
            edge_list.append({'source': node_id, 'target': hypernym_id, 'type': 'is-a'})
        
        # 上位概念を再帰的に処理 (深さ制限内で)
        if current_depth > -depth: # 階層を上に遡るため、マイナス方向に処理
            extract_hypernym_tree(hypernym, depth, max_hyponyms, current_depth - 1, node_list, edge_list)

    # --- 2. 下位概念（ヒポニミー）を抽出 ---
    # 下位概念は数が多すぎる場合があるため、制限を設ける
    hyponyms = synset.hyponyms()
    
    for i, hyponym in enumerate(hyponyms):
        if i >= max_hyponyms:
            # max_hyponyms を超えたら省略ノードを追加
            if current_depth < depth and node_id + "_omission" not in [n['id'] for n in node_list]:
                omission_id = node_id + "_omission"
                node_list.append({'id': omission_id, 'label': '...', 'depth': current_depth + 1})
                edge_list.append({'source': node_id, 'target': omission_id, 'type': 'omitted'})
            break
            
        hyponym_id = hyponym.name()
        # エッジを追加 (上位 -> 下位)
        if (node_id, hyponym_id) not in edge_list:
            edge_list.append({'source': hyponym_id, 'target': node_id, 'type': 'is-a'}) # 向きを統一
            
        # 下位概念を再帰的に処理
        extract_hypernym_tree(hyponym, depth, max_hyponyms, current_depth + 1, node_list, edge_list)
        
    return node_list, edge_list

# --- 実行 ---

# 中心的な単語（シノニムセット）を設定
# 例: 'dog' の名詞シノニムの最初のもの ('dog.n.01')
center_word = 'animal'
synsets = wn.synsets(center_word, pos=wn.NOUN)

if synsets:
    root_synset = synsets[0] 
    print(f"中心的なSynset: {root_synset.name()} ('{root_synset.definition()}')")

    # 階層の深さを設定
    DEPTH_LIMIT = 2
    MAX_HYPONYMS = 4
    
    # 階層構造を抽出
    nodes, edges = extract_hypernym_tree(
        root_synset, 
        depth=DEPTH_LIMIT, 
        max_hyponyms=MAX_HYPONYMS, 
        current_depth=0
    )

    # --- 結果の表示 ---
    print("\n" + "="*40)
    print(f"🌲 抽出されたWordNet階層構造 ({center_word})")
    print("="*40)
    
    print(f"ノード総数: {len(nodes)}")
    print(f"エッジ総数: {len(edges)}")
    
    print("\n--- ノードリストの例 ---")
    for node in nodes[:10]:
        print(f"ID: {node['id']}, Label: {node['label']}, Depth: {node['depth']}")
    
    print("\n--- エッジリストの例 ---")
    for edge in edges[:10]:
        # source -> target が (下位概念 -> 上位概念) の関係を表します
        print(f"Source: {edge['source']} -> Target: {edge['target']}, Type: {edge['type']}")
else:
    print(f"'{center_word}' に対応する名詞のSynsetが見つかりませんでした。")

中心的なSynset: animal.n.01 ('a living organism characterized by voluntary movement')
